In [ ]:
import time

from pydrake.geometry import StartMeshcat
from pydrake.geometry.optimization import Point
from pydrake.planning import GcsTrajectoryOptimization

from underactuated import running_as_notebook
from underactuated.uav_environment import (
    CONVEX_GCS_OPTION,
    NONLINEAR_GCS_OPTION,
    UavEnvironment,
)

### Generate UAV Environment

In [ ]:
meshcat = StartMeshcat()

In [ ]:
uav_env = UavEnvironment(seed=8)
regions, edges_between_regions = uav_env.compile()

if not running_as_notebook:
    CONVEX_GCS_OPTION.max_rounded_paths = 1
    NONLINEAR_GCS_OPTION.max_rounded_paths = 1

### Plan a Trajectory for the UAV

In [ ]:
# The maximum velocity limits for the skydio2
# were obtained from their website.
qDt_max = 16.0
# While the maximum acceleration are not publicly available, we assume
# an estimated thrust to weight ratio of something slightly greater than 2.
qDDt_max = 10.0

#### Baseline: Classical GCS
With the classical GCS formulation, we can enforce velocity limits, but no acceleration limits.
The duration transcription doesn't allow us to enforce higher order continuity constraints on the trajectory.
In the objective the path length and time will be minimized.

In [ ]:
gcs = GcsTrajectoryOptimization(3)
main = gcs.AddRegions(regions, edges_between_regions, order=6, h_min=0, h_max=20)
source = gcs.AddRegions(
    [Point(uav_env.DEFAULT_START)], order=0, h_min=0, h_max=0, name="source"
)
target = gcs.AddRegions(
    [Point(uav_env.DEFAULT_GOAL)], order=0, h_min=0, h_max=0, name="target"
)
source_to_main = gcs.AddEdges(source, main)
main_to_target = gcs.AddEdges(main, target)

source_to_main.AddZeroDerivativeConstraints(1)
main_to_target.AddZeroDerivativeConstraints(1)
source_to_main.AddZeroDerivativeConstraints(2)
main_to_target.AddZeroDerivativeConstraints(2)

gcs.AddVelocityBounds(3 * [-qDt_max], 3 * [qDt_max])

gcs.AddTimeCost()
gcs.AddPathLengthCost()

start_time = time.time()
baseline_traj, result = gcs.SolvePath(source, target, CONVEX_GCS_OPTION)
print(
    f"Is successful: {result.is_success()} in {round(time.time() - start_time, 3)} seconds."
)
print(
    f"Total trajectory time: {round(baseline_traj.end_time() - baseline_traj.start_time(), 3)} seconds."
)

#### Nonlinear GCS: Shortest Path
With the nonlinear extension, we can enforce acceleration limits in addition to the velocity limits.
Further, we can enforce higher continuity constraints, here we will enforce continuity up to the 4th derivative.
In the objective the path length and time will be minimized.

In [ ]:
gcs = GcsTrajectoryOptimization(3)
main = gcs.AddRegions(regions, edges_between_regions, order=6, h_min=0, h_max=20)
source = gcs.AddRegions(
    [Point(uav_env.DEFAULT_START)], order=0, h_min=0, h_max=0, name="source"
)
target = gcs.AddRegions(
    [Point(uav_env.DEFAULT_GOAL)], order=0, h_min=0, h_max=0, name="target"
)
source_to_main = gcs.AddEdges(source, main)
main_to_target = gcs.AddEdges(main, target)

source_to_main.AddZeroDerivativeConstraints(1)
main_to_target.AddZeroDerivativeConstraints(1)
source_to_main.AddZeroDerivativeConstraints(2)
main_to_target.AddNonlinearDerivativeBounds(3 * [0], 3 * [0], 2)

gcs.AddContinuityConstraints(1)  # Velocity Continuity
gcs.AddContinuityConstraints(2)  # Acceleration Continuity
gcs.AddContinuityConstraints(3)  # Jerk Continuity
gcs.AddContinuityConstraints(4)  # Snap Continuity

gcs.AddVelocityBounds(3 * [-qDt_max], 3 * [qDt_max])
gcs.AddNonlinearDerivativeBounds(3 * [-qDDt_max], 3 * [qDDt_max], 2)

gcs.AddTimeCost()
gcs.AddPathLengthCost()

start_time = time.time()
shortest_path_traj, result = gcs.SolvePath(source, target, NONLINEAR_GCS_OPTION)
print(
    f"Is successful: {result.is_success()} in {round(time.time() - start_time, 3)} seconds."
)
if result.is_success():
    print(
        f"Total trajectory time: {round(shortest_path_traj.end_time() - shortest_path_traj.start_time(), 3)} seconds."
    )

#### Nonlinear GCS: Minimum Snap
This problem is similar to the shortest path problem, but the objective is to minimize the snap.

In [ ]:
## NOTE: this example is waiting on AddNormalizedPathDerivativeCost landing in Drake.

# gcs = GcsTrajectoryOptimization(3)
# main = gcs.AddRegions(regions, edges_between_regions, order=6, h_min=1e-4, h_max=20)
# source = gcs.AddRegions(
#     [Point(uav_env.DEFAULT_START)], order=0, h_min=0, h_max=0, name="source"
# )
# target = gcs.AddRegions(
#     [Point(uav_env.DEFAULT_GOAL)], order=0, h_min=0, h_max=0, name="target"
# )
# source_to_main = gcs.AddEdges(source, main)
# main_to_target = gcs.AddEdges(main, target)

# source_to_main.AddVelocityBounds(3 * [0], 3 * [0])
# main_to_target.AddVelocityBounds(3 * [0], 3 * [0])
# source_to_main.AddNonlinearDerivativeBounds(3 * [0], 3 * [0], 2)
# main_to_target.AddNonlinearDerivativeBounds(3 * [0], 3 * [0], 2)

# gcs.AddContinuityConstraints(1)  # Velocity Continuity
# gcs.AddContinuityConstraints(2)  # Acceleration Continuity
# gcs.AddContinuityConstraints(3)  # Jerk Continuity
# gcs.AddContinuityConstraints(4)  # Snap Continuity

# gcs.AddVelocityBounds(3 * [-qDt_max], 3 * [qDt_max])
# gcs.AddNonlinearDerivativeBounds(3 * [-qDDt_max], 3 * [qDDt_max], 2)

# # Minimum snap cost.
# gcs.AddNormalizedPathDerivativeCost(4)

# start_time = time.time()
# minimum_snap_traj, result = gcs.SolvePath(source, target, NONLINEAR_GCS_OPTION)
# print(
#     f"Is successful: {result.is_success()} in {round(time.time() - start_time, 3)} seconds."
# )
# print(
#     f"Total trajectory time: {round(minimum_snap_traj.end_time() - minimum_snap_traj.start_time(), 3)} seconds."
# )

### Visualize the Trajectories

In [ ]:
if result.is_success():
    uav_env.animate_trajectory(
        meshcat,
        [baseline_traj, shortest_path_traj],
        #    [baseline_traj, shortest_path_traj, minimum_snap_traj],
        fly_in_sequence=True,
    )